In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

In [2]:
import os, sys, time

In [3]:
import ROOT

Welcome to JupyROOT 6.10/09


In [4]:
from importlib import import_module
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor

In [5]:
from PhysicsTools.NanoAODTools.postprocessing.framework.datamodel import Collection, Object
from PhysicsTools.NanoAODTools.postprocessing.framework.eventloop import Module

In [ ]:
class testing(Module):
    def __init__(self):
        pass
    def beginJob(self):
        pass
    def endJob(self):
        pass
    #def beginFile(self, inputFile, outputFile):
    #def beginFile(self):
        #pass
    #def endFile(self, inputFile, outputFile):
    #def endFile(self):
        #pass
    def analyze(self, event):
        """process event, return True (go to next module) or False (fail, go to next event)"""
        
        HLT = Object(event, "HLT") 
        Filters = Object(event, "Flag") 
        for trig in passTrig:
            print("HLT_" + str(trig) + " Trigger: " + str(getattr(HLT, trig)) )
        passFilter=["HBHENoiseFilter", "HBHENoiseIsoFilter", "EcalDeadCellTriggerPrimitiveFilter", 
                    "globalSuperTightHalo2016Filter", "goodVertices", "METFilters"]
        for fltr in passFilter:
            print("Flag_" + str(fltr) + " Filter: " + str(getattr(Filters, fltr)))
            
        electrons = Collection(event, "Electron")
        photons = Collection(event, "Photon")
        muons = Collection(event, "Muon")
        taus = Collection(event, "Tau")
        jets = Collection(event, "Jet")
        met = Object(event, "MET")
        run = getattr(event, "run")
        lumi = getattr(event, "lumi") #WELL THERE ZHE BLOWS! I tried filtering by event instead of lumisection... fail
        evt = getattr(event, "event")
        PV = Object(event, "PV")
        SV = Collection(event, "SV")
        genParts = Collection(event, "GenPart")
        genJets = Collection(event, "GenJet")
        genFatJets = Collection(event, "GenJetAK8")
        eventSum = ROOT.TLorentzVector()
        print("Run: {0:>8d} Event: {1:>8d}".format(run, evt))
        print("PV  X: {0: >5.3f} Y: {1: >5.3f} Z: {2:5.3f} nDoF: {3: >5f} Chi^2: {4: >5.3f}".format(
            PV.x,PV.y, PV.z, PV.ndof, PV.chi2))
        if len(SV) > 0:   
            print("nSV: {0: >3d} SV[0] Decay Length:{1: >5.3f}".format(len(SV), SV[0].dlen ))
        else:
            print("nSV: {0: >3d}".format(len(SV)))
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Muon", "Pt", "Eta", "Phi"))
        for nm, lep in enumerate(muons) :
            eventSum += lep.p4()
            #format_spec ::=  [[fill]align][sign][#][0][width][,][.precision][type]
            print("{0:*<5d} {1:>10.4f} {2:>+10.3f} {3:>+10.3f}".format(nm+1, lep.pt, lep.eta, lep.phi))
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Electron", "Pt", "Eta", "Phi"))
        for ne, lep in enumerate(electrons) :
            eventSum += lep.p4()
            print("{0:*^5d} {1:>10.4f} {2:> 10.3f} {3:> 10.3f}".format(ne+1, lep.pt, lep.eta, lep.phi))
        #for j in filter(self.jetSel,jets):
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Jet", "Pt", "Eta", "Phi"))
        for nj, j in enumerate(jets):
            eventSum += j.p4()
            print("{0: >5d} {1:>10.4f} {2:>-10.3f} {3:>-10.3f}".format(nj+1, j.pt, j.eta, j.phi))
        passTrig=["PFMETNoMu90_PFMHTNoMu90_IDTight"]
        print("Event Mass: {:<10.4f}\n".format(eventSum.M()))
        return True


In [7]:
#preselection="nMuon > 0 && nElectron > 0"
preselection=None
filePrefix = "root://cms-xrd-global.cern.ch/"
inputList = open("../data/Run2016/Run2016B_SM", "r")
files=[]
for line in inputList:
    files.append(filePrefix + str(line))
#for file in files:
#    print(file)

In [ ]:
p=PostProcessor(".",files,cut=preselection,branchsel=None,modules=[testing()],friend=False,postfix="TrigAndJson", 
                jsonInput=None,noOut=True,justcount=True,provenance=False,
                haddFileName=None,fwkJobReport=False,histFileName=None,
                histDirName=None, outputbranchsel=None)

In [ ]:
p.run()